In [97]:
import pandas as pd

class LL1:
    def __init__(self, grammar):
        self.grammar = grammar
        self.non_terminals = list(grammar.keys())
        self.terminals = self.get_terminals()
        self.first = self.compute_first()
        self.follow = self.compute_follow()
        self.table = self.construct_table()

    def get_terminals(self):
        terminals = set()
        for productions in self.grammar.values():
            for prod in productions:
                for char in prod:
                    if not char.isupper():
                        terminals.add(char)
        return list(terminals)

    def compute_first(self):
        first = {}
        for non_terminal in self.non_terminals:
            first[non_terminal] = list()

        for non_terminal in self.non_terminals:
            first[non_terminal].append(self.find_first(non_terminal))
        return first

    def find_first(self, non_terminal):
        productions = self.grammar[non_terminal]
        
        for production in productions:
            first_element = production[0]

            if first_element.isupper():
                return self.find_first(first_element)
            else:
                return first_element
    # Follow()

grammar = {
    'E': ['TX'],
    'X': ['+TX'],
    'T': ['-FY='],
    'Y': ['*FY'],
    'F': ['(E)']
}

ll1_parser = LL1(grammar)
ll1_parser.construct_table()


,Non Terminal,First(),Follow()
0,E,-,"{), $}"
1,X,+,"{), $}"
2,T,-,{+}
3,Y,*,{=}
4,F,(,{*}
